In [ ]:
!git clone -b feat/add_vectordb https://github.com/collaborative-earth/aicacia.git
%cd aicacia/vectordb

Cloning into 'aicacia'...
remote: Enumerating objects: 1254, done.
remote: Counting objects: 100% (615/615), done.
remote: Compressing objects: 100% (301/301), done.
remote: Total 1254 (delta 400), reused 490 (delta 312), pack-reused 639 (from 1)
Receiving objects: 100% (1254/1254), 10.36 MiB | 7.35 MiB/s, done.
Resolving deltas: 100% (652/652), done.
/content/aicacia/vectordb


In [ ]:
from google.colab import drive, output
output.enable_custom_widget_manager()
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install llama-index
%pip install sqlite
%pip install llama-index-embeddings-huggingface
%pip install llama-index-vector-stores-qdrant

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.5/264.5 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement sqlite (from versions: none)
ERROR: No matching distribution found for sqlite
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s 

In [ ]:
#!python main.py \
#  -c "aicaica" \
#  -d  "/content/drive/Shareddrives/Aicacia MicroLab/Scraped Data/WRI"

In [ ]:
import argparse
import os
import sys
from glob import glob

from llama_index.core import SimpleDirectoryReader
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import HTMLNodeParser, MarkdownNodeParser
from tqdm import tqdm
sys.path.append("..")
from utils import *
from custom_parsers import *
from qdrant_client import QdrantClient
sys.path.append("..")
#from finetuning.src.node_parsers.tei_parser import TEINodeParser


def parse_args():
    parser = argparse.ArgumentParser(description="VectorDB Arguments")
    parser.add_argument('-c', '--collection', help="Collection Name")
    parser.add_argument('-d', '--input-dir', help="Input Directory")
    args = parser.parse_args()
    return args


extraction_functions = {
        #'.md': MarkdownNodeParser(),
        '.html': HTMLNodeParser(),
        #'.pdf': MarkdownNodeParser(),
        '.xml': TEINodeParser(),
        '.json': MyJSONNodeParser()
    }

    #args = parse_args()
collection_name = "aicacia"#args.collection
input_dir ="/content/drive/Shareddrives/Aicacia MicroLab/Scraped Data/WRI" #args.input_dir

config = load_config('config.yaml')
text_splitter = create_text_splitter(config)
embed_model = create_embedding_class(config)
qdrant_client = QdrantClient(
    url="https://dfb56621-0186-46e4-975a-3d1b4f0f6db3.us-west-1-0.aws.cloud.qdrant.io",
    api_key= YOUR_APIKEY
)
vector_store = QdrantVectorStore(client=qdrant_client, collection_name="aicacia")
#vector_store = create_vectordb_client(config, collection_name)
metadata_df = read_db(input_dir)[['id', 'title', 'doi', 'authors',
                                'corpus_name', 'sources', 'location',
                                'sourced_date', 'revision_date',
                                'provided_tags', 'generated_tags','metadata']]
metadata_df["extracted_file_name"] = metadata_df["metadata"].apply(extract_file_name)

for ext, file_parser in extraction_functions.items():

    try:
        reader = SimpleDirectoryReader(input_dir=input_dir,
                                        recursive=True,
                                        required_exts=[ext],
                                        file_extractor = create_file_extractor(config),
                                        num_files_limit=100)
    except Exception as e:
        if 'No files found' in str(e):
            continue
        else:
            raise Exception(e)

    docs = []
    input_files = reader.input_files
    for file_path in tqdm(input_files):
        reader.input_files = [file_path]
        file_name = str(file_path).split('/')[-1].split('.')[-2]
        try:
            file_metadata = metadata[metadata['id'] == file_name].iloc[0].to_dict()
        except:
            file_metadata = {}

        # Load the document with the file extractor
        loaded_docs = reader.load_data()

        # Enrich each document with metadata
        for doc in loaded_docs:
            doc.metadata.update(file_metadata)
            doc.excluded_llm_metadata_keys = list(doc.metadata.keys())
            doc.excluded_embed_metadata_keys = list(doc.metadata.keys())
        docs.extend(loaded_docs)


    pipeline = IngestionPipeline(transformations=[file_parser, text_splitter, MetadataCleanupTransformation(),embed_model], vector_store=vector_store)
    nodes = pipeline.run(documents=docs, show_progress=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]


100%|██████████| 58/58 [00:13<00:00,  4.29it/s]


Parsing nodes:   0%|          | 0/58 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/3122 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/3768 [00:00<?, ?it/s]

In [ ]:
node_url = "https://dfb56621-0186-46e4-975a-3d1b4f0f6db3.us-west-1-0.aws.cloud.qdrant.io"
snapshot_info = qdrant_client.create_snapshot(collection_name="aicacia")
snapshot_url = f"{node_url}/collections/aicacia/snapshots/{snapshot_info.name}"


In [17]:
# DOwnload the snapshot
import requests
snapshot_name = os.path.basename(snapshot_url)
local_snapshot_path = os.path.join(".", snapshot_name)

response = requests.get(
    snapshot_url, headers={"api-key": QDRANT_API_KEY}
)
with open(local_snapshot_path, "wb") as f:
    response.raise_for_status()
    f.write(response.content)



In [18]:
# Try to import new collection
snapshot_name = os.path.basename(local_snapshot_path)
requests.post(
    f"{node_url}/collections/test_collection_import/snapshots/upload?priority=snapshot",
    headers={
        "api-key": QDRANT_API_KEY,
    },
    files={"snapshot": (snapshot_name, open(local_snapshot_path, "rb"))},
)

<Response [200]>

In [22]:
response = requests.get(f"{node_url}/collections/aicacia", headers={
        "api-key": QDRANT_API_KEY,
    })

if response.status_code == 200:
    data = response.json()
    point_count = data.get("result", {}).get("points_count", 0)
    print(f"Number of entries: {point_count}")
else:
    print(f"Error: {response.status_code}, {response.text}")

Number of entries: 3819


In [16]:
qdrant_client.scroll('aicacia')[0]

[Record(id='000196f3-843c-49c8-8ae6-333258ae2abe', payload={'file_name': 'aligning-ndcs-1-5-degc-net-zero-low-emissions-development-strategies.json', 'file_type': 'application/json', 'file_size': 80607, 'creation_date': '2025-02-25', 'last_modified_date': '2025-02-25', '_node_content': '{"id_": "000196f3-843c-49c8-8ae6-333258ae2abe", "embedding": null, "metadata": {"file_name": "aligning-ndcs-1-5-degc-net-zero-low-emissions-development-strategies.json", "file_type": "application/json", "file_size": 80607, "creation_date": "2025-02-25", "last_modified_date": "2025-02-25"}, "excluded_embed_metadata_keys": ["file_path", "file_name", "file_type", "file_size", "creation_date", "last_modified_date"], "excluded_llm_metadata_keys": ["file_path", "file_name", "file_type", "file_size", "creation_date", "last_modified_date"], "relationships": {"1": {"node_id": "b92fefb7-78e6-463c-8081-cce618bfe01b", "node_type": "4", "metadata": {"file_path": "/content/drive/Shareddrives/Aicacia MicroLab/Scraped 

In [ ]:
qdrant_client.scroll(collection_name=collection_name)

([Record(id='0336718f-f028-4a2f-8045-39f901c1801c', payload={'file_name': 'aligning-ndcs-1-5-degc-net-zero-low-emissions-development-strategies.json', 'file_type': 'application/json', 'file_size': 80607, 'creation_date': '2025-02-25', 'last_modified_date': '2025-02-25', '_node_content': '{"id_": "0336718f-f028-4a2f-8045-39f901c1801c", "embedding": null, "metadata": {"file_name": "aligning-ndcs-1-5-degc-net-zero-low-emissions-development-strategies.json", "file_type": "application/json", "file_size": 80607, "creation_date": "2025-02-25", "last_modified_date": "2025-02-25"}, "excluded_embed_metadata_keys": ["file_path", "file_name", "file_type", "file_size", "creation_date", "last_modified_date"], "excluded_llm_metadata_keys": ["file_path", "file_name", "file_type", "file_size", "creation_date", "last_modified_date"], "relationships": {"1": {"node_id": "ffbe71c7-55a1-400f-b76e-fb3c3445a3bb", "node_type": "4", "metadata": {"file_path": "/content/drive/Shareddrives/Aicacia MicroLab/Scraped